# Simple Model
In our simpler model, we will just model each post as posting about a story coming from one of three groups:
- Factual, Disputed Story
- Fake, Disputed Story
- Corrective Story

Have found this to be a useful resource for a hierarchcal model example: https://github.com/pyro-ppl/pyro/blob/dev/examples/baseball.py
As well as https://pyro.ai/examples/forecasting_iii.html

# To start, we will use dummy data

In [1]:
import pandas as pd
import torch
import pyro
from pyro.infer import MCMC, NUTS
import pyro.distributions as dist
from pyro.distributions.util import scalar_like
from torch.distributions import constraints

In [2]:
pyro.enable_validation(__debug__)
pyro.set_rng_seed(0)

Let's try using non-rectangular data to do the same thing. (Get rid of this type level and make it into a categorical variable instead!)

In [3]:
# Post-Level Data
p_data = torch.Tensor([[1, 100, 0, 1000], [1, 250, 0, 3000], [1, 125, 0, 1500], [1, 150, 0, 1500],
                       [1, 50,  1, 800],  [1, 100, 1, 2500], [1, 150, 1, 1600], [1, 125, 1, 1200],
                       [1, 20,  2, 300],  [1, 40,  2,  500], [1, 30,  2, 1000], [1, 35,  2, 600]])
p_data = p_data.transpose(0,1)
# dim 0: post-level vars: (bias, commentsFirstHour, type, Engagement) 
# dim 1: obs (post)

In [4]:
p_data.numpy().astype(int)

array([[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1],
       [ 100,  250,  125,  150,   50,  100,  150,  125,   20,   40,   30,
          35],
       [   0,    0,    0,    0,    1,    1,    1,    1,    2,    2,    2,
           2],
       [1000, 3000, 1500, 1500,  800, 2500, 1600, 1200,  300,  500, 1000,
         600]])

In [5]:
# Type-Level Data
t_data = torch.Tensor([[1], [1], [1]])
t_data = t_data.transpose(0,1)
# dim 0: type-level vars: (Just bias for now)
# dim 1: type (0: Fake, 1: Fact, 2: Corrective)

In [6]:
t_data.shape

torch.Size([1, 3])

In [7]:
p_data.shape

torch.Size([4, 12])

In [8]:
y = p_data[-1,:]
p_data = p_data[:-1,:]

In [9]:
p_data

tensor([[  1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   1.],
        [100., 250., 125., 150.,  50., 100., 150., 125.,  20.,  40.,  30.,  35.],
        [  0.,   0.,   0.,   0.,   1.,   1.,   1.,   1.,   2.,   2.,   2.,   2.]])

In [10]:
p_data.shape

torch.Size([3, 12])

In [11]:
y

tensor([1000., 3000., 1500., 1500.,  800., 2500., 1600., 1200.,  300.,  500.,
        1000.,  600.])

Type-Level Regressions
phi_0,t = eta_0^T t_data + epsilon_0
phi_1,t = eta_1^T t_data + epsilon_1

Post-Level Regression
y_pt = phi_0,t * bias + phi_1,t * first_hour_comments

In [12]:
# x is a 3D tensor
def model(t_data, p_data, y):
    num_t_indeps, num_types = t_data.shape
    num_p_indeps, num_posts = p_data.shape
    
    num_p_indeps -= 1 # The last p_indep is just the type.
    
    # construct necessary plates over each level
    type_plate = pyro.plate("type", num_types)
    post_plate = pyro.plate("post", num_posts)
    t_indep_plate = pyro.plate("t_indep", num_t_indeps)
    p_indep_plate = pyro.plate("p_indep", num_p_indeps)
    
    
    # type-level regression variables (shared across all types)
    # one coef for each type-level indep var for each of num_p_indeps type-level regressions
    # each row are the coefficients for a different type-level regression
    # eta in proposal
    type_level_coefs = torch.empty((num_p_indeps, num_t_indeps,)) # (2,1)
    for pi in p_indep_plate:
        for ti in t_indep_plate:
            type_level_coefs[pi, ti] = pyro.sample(f"type_level_coef_{ti}_on_pi_{pi}", dist.Normal(0, 10))
        

    # Run a type-level regression for the coef on each post-level variable
    # phi in proposal
    type_varying_post_level_coefs = torch.empty((num_p_indeps, num_types,)) # (2,3)
    for pi in p_indep_plate:
        for t in type_plate:
            type_varying_post_level_coef_mu = torch.dot(type_level_coefs[pi,:], t_data[:,t])

            # get the restulting type-varying post-level coefficient
            type_varying_post_level_coefs[pi,t] = pyro.sample(f"type_{t}_post_level_coef_{pi}", 
                                                              dist.Normal(type_varying_post_level_coef_mu, 10.))
    
    # for each post, use the correct set of coefficients to run our post-level regression
    for p in post_plate:
        t = int(p_data[-1,p])
        
        coefs = type_varying_post_level_coefs[:,t] # phi for this post.
        indeps = p_data[:-1,p] # indep vars for this post
        # calculate the mean
        mu = torch.dot(coefs, indeps)
        
        # sample
        pyro.sample(f"obs_{t}_{p}", dist.Normal(mu, 1000.), obs=y[p])
        
            

In [13]:
# # x is a 3D tensor
# def guide(t_data, p_data, y):
#     num_t_indeps, num_types = t_data.shape
#     num_p_indeps, num_posts = p_data.shape
    
#     num_p_indeps -= 1 # The last p_indep is just the type.
    
#     # construct necessary plates over each level
#     type_plate = pyro.plate("type", num_types)
#     post_plate = pyro.plate("post", num_posts)
#     t_indep_plate = pyro.plate("t_indep", num_t_indeps)
#     p_indep_plate = pyro.plate("p_indep", num_p_indeps)
    
    
    
#     type_level_coef_locs = torch.empty((num_p_indeps, num_t_indeps,))   # (2,1)
#     type_level_coef_scales = torch.empty((num_p_indeps, num_t_indeps,)) # (2,1)
    
#     for pi in p_indep_plate:
#         for ti in t_indep_plate:
#             type_level_coef_locs[pi, ti] = pyro.param(f"type_level_coef_{ti}_on_pi_{pi}_loc", 
#                                                       torch.Tensor(0.))
#             type_level_coef_scales[pi, ti] = pyro.param(f"type_level_coef_{ti}_on_pi_{pi}_scale", 
#                                                         torch.Tensor(1.), 
#                                                         constraint=constraints.positive)
    
#     # type-level regression variables (shared across all types)
#     # one coef for each type-level indep var for each of num_p_indeps type-level regressions
#     type_level_coefs = torch.empty((num_p_indeps, num_t_indeps,)) # (2,1)
#     for pi in p_indep_plate:
#         for ti in t_indep_plate:
#             type_level_coefs[pi, ti] = pyro.param(f"type_level_coef_{ti}_on_pi_{pi}", dist.Normal(type_level_coef_locs[pi,ti], type_level_coef_scales[pi,ti]))
        

#     type_varying_post_level_coef_scales = torch.empty((num_p_indeps, num_types,))
#     for pi in p_indep_plate:
#         for t in type_plate:
#             type_varying_post_level_coef_scales[pi,t] = pyro.param(f"type_{t}_post_level_coef_{pi}_scale", torch.Tensor(1.), constraint=constraints.positive)
    
#     # Run a type-level regression for the coef on each post-level variable
#     type_varying_post_level_coefs = torch.empty((num_p_indeps, num_types,))
#     for pi in p_indep_plate:
#         for t in type_plate:
#             type_varying_post_level_coef_mu = torch.dot(type_level_coefs[pi,:], t_data[:,t])
# #             type_varying_post_level_coef_std = pyro.sample(f"type_{t}_post_level_coef_{pi}_std", dist.Uniform(0., 10.))

#             # get the restulting type-varying post-level coefficient
#             type_varying_post_level_coefs[pi,t] = pyro.param(f"type_{t}_post_level_coef_{pi}", dist.Normal(type_varying_post_level_coef_mu, type_varying_post_level_coef_scales[pi,t]))
        

In [14]:
nuts_kernel = NUTS(model)

mcmc = MCMC(nuts_kernel, num_samples=2000, warmup_steps=1000)
mcmc.run(t_data, p_data, y)

hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}

Sample: 100%|██████████| 3000/3000 [04:42, 10.61it/s, step size=4.24e-01, acc. prob=0.917]


In [15]:
TYPES = ["Fake", "Fact", "Corrective"]
# Utility function to print latent sites' quantile information.
def summary_types(samples):
    site_stats = {}
    i = 0
    for site_name, values in samples.items():
#         values = values.reshape((values.shape[0], values.shape[1]))
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles=[.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
#         site_stats[site_name]["type"] = TYPES
        i += 1
    return site_stats

In [16]:
hmc_samples

{'type_level_coef_0_on_pi_0': array([17.530804 ,  5.76418  ,  6.4192877, ..., -7.6067123,  8.856096 ,
         3.9015255], dtype=float32),
 'type_level_coef_0_on_pi_1': array([ 1.5550346,  5.4955816,  8.0096855, ..., 15.52832  , 10.444261 ,
         2.117679 ], dtype=float32),
 'type_0_post_level_coef_0': array([ 20.885597 ,  15.160886 ,   8.555441 , ..., -16.150133 ,
         27.244688 ,  -7.2345715], dtype=float32),
 'type_1_post_level_coef_0': array([  3.3984413,  14.195103 ,  13.916082 , ..., -11.696006 ,
         15.135899 ,   8.179921 ], dtype=float32),
 'type_2_post_level_coef_0': array([17.12295  , 14.363754 ,  5.2270055, ..., -3.1877165,  6.7148495,
         9.67814  ], dtype=float32),
 'type_0_post_level_coef_1': array([11.69773  ,  9.559046 ,  8.236295 , ..., 12.13768  , 11.7400465,
         4.572459 ], dtype=float32),
 'type_1_post_level_coef_1': array([11.235755, 16.03295 ,  8.515057, ..., 10.538765, 22.0059  ,
        13.829964], dtype=float32),
 'type_2_post_level_coef_1

In [17]:

for site, values in summary_types(hmc_samples).items():
    print("Coefficient: {}".format(site))
    print(values, "\n")

Coefficient: type_level_coef_0_on_pi_0
       mean       std         5%       25%       50%       75%       95%
0 -0.253649  10.16328 -17.307125 -6.930449 -0.351429  6.651258  16.59324 

Coefficient: type_level_coef_0_on_pi_1
       mean       std        5%       25%       50%        75%        95%
0  8.990003  5.964278 -0.495992  4.967722  8.942105  13.024718  18.841787 

Coefficient: type_0_post_level_coef_0
       mean        std         5%        25%       50%      75%        95%
0 -0.133035  14.557309 -23.881184 -10.030286 -0.179503  9.68175  24.037552 

Coefficient: type_1_post_level_coef_0
       mean        std         5%        25%       50%       75%        95%
0 -0.131228  14.262767 -22.987999 -10.091269 -0.196285  9.248511  23.524866 

Coefficient: type_2_post_level_coef_0
      mean        std        5%       25%       50%        75%        95%
0  0.28256  14.214364 -23.58259 -8.972426  0.493736  10.041537  23.089745 

Coefficient: type_0_post_level_coef_1
        mean    